In [54]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
from pandas import DataFrame, Series  # for convenience
from scipy.misc import imsave
import pims
import trackpy as tp
from pims import Frame
import math

################################################
# Read csv data of tracking
################################################
BR = [[5.7149E-08,1.1882E-05,-2.4321E-03],[8.8831E-06,-5.3566E-07,-3.8744E-02],[-1.6469E-03,3.3774E-02,9.9867E-01]]
normal = np.max(BR)
MatrixL = np.array(BR)/normal
MatrixR = MatrixL.transpose()
dfL = pd.read_csv('frameL.csv')
dfR = pd.read_csv('frameR.csv')

In [55]:
dfL.head()

,frame,x,y,mass,size,ecc,signal,raw_mass,ep,frame.1,particle
0,0,242.461330,88.819362,773.857891,1.747439,0.096445,68.220030,1319.0,0.000431,0,2.0
1,1,241.513562,87.584690,718.830824,1.770769,0.125445,61.094121,1249.0,0.000327,1,2.0
2,2,240.583741,86.028404,748.729596,1.825384,0.123629,59.399703,1340.0,0.000166,2,2.0
3,3,239.615206,84.856745,780.016490,1.785300,0.154906,64.730158,1384.0,0.000201,3,2.0
4,4,238.656832,83.700862,719.448382,1.794858,0.192733,56.171185,1288.0,0.000624,4,2.0


In [68]:
#dfL.head()
#df1 = dfL[dfL['particle']==2]
#frame_mid_L = int(np.median(df1['frame.1']))
df1 = df1[df1['frame.1'] == frame_mid_L]
x_mid = float(df1['x'])
y_mid = float(df1['y'])
ipL = [x_mid,y_mid,1]

In [ ]:
off = np.ones((1,1))
pair=[]
for iparticleL in dfL['particle'].unique():
    df1 = dfL[dfL['particle']==iparticleL]
    frame_mid_L = int(np.median(df1['frame.1']))
    df1 = df1[df1['frame.1'] == frame_mid_L]
    x_mid = float(df1['x'])
    y_mid = float(df1['y'])
    MinInd = 0
    Min = 10000
    for iparticleR in dfR['particle'].unique():
        df2 = dfR[dfR['particle']==iparticleR]
        frame_max_R = np.max(df2['frame.1'])
        if(frame_max_R>=frame_mid_L):
            df2 = df2[df2['frame.1'] == frame_mid_L]
            if (df2.empty)==0:
                #print("")
                #print("Frame ",iparticle," is empty.")
            #else:
            #print(df2.head())
                x_mid = float(df2['x'])
                y_mid = float(df2['y'])
                ipR = [x_mid,y_mid,1]
                LineL = np.dot(ipR,MatrixL) #points in the right frame and the corresponding epipolar lines in the left frame
                LineR = np.dot(ipL,MatrixR) #points in the right frame and the corresponding epipolar lines in the left frame
                dlr = math.fabs(np.dot(ipL,LineL))/math.sqrt(LineL[0]*LineL[0]+LineL[1]*LineL[1])
                drl = math.fabs(np.dot(ipR,LineR))/math.sqrt(LineR[0]*LineR[0]+LineR[1]*LineR[1])
                FF = dlr+drl
                #print(iparticle,FF)
                if(FF<Min):
                    MinInd = iparticleR
                    Min = FF        
        #else:
            #print("No this frame!")
        #    print("")
    pair1 = [iparticleL,MinInd]
    pair.append(pair1)
    print(iparticleL, MinInd,Min)

(2.0, 7.0, 2.6920716003968144)
(3.0, 4.0, 5.590671591918162)
(4.0, 7.0, 2.6920716003968144)
(6.0, 5.0, 3.999974972874339)
(7.0, 8.0, 2.9255124435620887)
(8.0, 305.0, 14.526403885028255)
(9.0, 5.0, 5.6880697445065405)
(13.0, 152.0, 0.1634662647611704)
(15.0, 5.0, 0.10536825838637284)
(17.0, 5.0, 2.3749670565038676)
(19.0, 173.0, 3.102581552784188)
(22.0, 4.0, 0.6297162471083342)
(23.0, 48.0, 0.2108639073046902)
(25.0, 5.0, 0.10536825838637284)
(28.0, 279.0, 2.233066820612792)
(29.0, 8.0, 2.9255124435620887)
(32.0, 3.0, 14.446915152546678)
(34.0, 5.0, 2.2663413981069698)
(38.0, 3.0, 14.446915152546678)
(39.0, 3.0, 3.3396581371445873)
(45.0, 3.0, 5.375148572559235)
(47.0, 173.0, 6.1749208354453335)
(49.0, 18.0, 4.570358187000965)
(50.0, 4.0, 5.590671591918162)
(51.0, 59.0, 1.1411550778859865)
(54.0, 48.0, 0.2108639073046902)
(55.0, 305.0, 0.8874193614328458)
(56.0, 59.0, 1.1411550778859865)
(58.0, 5.0, 5.6880697445065405)
(60.0, 94.0, 1.0569406505222978)
(62.0, 7.0, 0.6384823884603077)
(6

In [ ]:
df = pd.DataFrame(pair,columns = ['Left','Right'])
df.to_csv('Pair.csv')